# 🚀 Kaggle Remote VS Code Setup

Connect VS Code to this Kaggle notebook remotely using zrok tunnel.

## Instructions:
1. **Edit the configuration** in Cell 1 below
2. **Run Cell 1** to set up SSH and zrok
3. **Run Cell 2** to start the tunnel (keep it running!)
4. **Copy the hostname** from Cell 2 output
5. **Use the hostname** on your local machine to connect

---

## ⚙️ Cell 1: Configuration & Setup

**Edit the configuration below, then run this cell.**

In [ ]:
# ╔═══════════════════════════════════════════════════════════════╗
# ║                    CONFIGURATION                              ║
# ║               ⚠️  EDIT THESE VALUES  ⚠️                        ║
# ╚═══════════════════════════════════════════════════════════════╝

ZROK_TOKEN = "YOUR_ZROK_TOKEN_HERE"      # Get from https://zrok.io
SSH_PASSWORD = "kaggle123"               # SSH login password
GITHUB_USERNAME = ""                     # Optional: for SSH key auth (leave empty to skip)

# Tunnel mode: "public" or "private"
# - PUBLIC:  Direct SSH (ssh root@hostname.share.zrok.io)
# - PRIVATE: Requires zrok on client (zrok access private <token>)
TUNNEL_MODE = "public"

# ╔═══════════════════════════════════════════════════════════════╗
# ║                    DO NOT EDIT BELOW                          ║
# ╚═══════════════════════════════════════════════════════════════╝

import os
import subprocess

def run(cmd, show=True):
    """Execute shell command"""
    if show:
        print(f"$ {cmd}")
    return os.system(cmd)

def main():
    print("=" * 60)
    print("🚀 Kaggle Remote VS Code Setup")
    print(f"   Tunnel Mode: {TUNNEL_MODE.upper()}")
    print("=" * 60)
    
    # Validate config
    if ZROK_TOKEN == "YOUR_ZROK_TOKEN_HERE":
        print("\n❌ ERROR: Please set your ZROK_TOKEN!")
        print("   Get one at https://zrok.io")
        return
    
    # Step 1: Install dependencies
    print("\n[1/5] 📦 Installing dependencies...")
    run("apt-get update -qq", show=False)
    run("apt-get install -y openssh-server > /dev/null 2>&1", show=False)
    print("      ✓ openssh-server installed")
    run("curl -sSf https://get.zrok.io 2>/dev/null | bash", show=False)
    print("      ✓ zrok installed")
    
    # Step 2: Configure SSH
    print("\n[2/5] 🔐 Configuring SSH server...")
    os.makedirs("/run/sshd", exist_ok=True)
    os.makedirs("/root/.ssh", exist_ok=True)
    os.chmod("/root/.ssh", 0o700)
    
    sshd_config = """Port 22
PermitRootLogin yes
PasswordAuthentication yes
PubkeyAuthentication yes
X11Forwarding yes
ClientAliveInterval 60
ClientAliveCountMax 3
Subsystem sftp /usr/lib/openssh/sftp-server
"""
    with open("/etc/ssh/sshd_config", "w") as f:
        f.write(sshd_config)
    print("      ✓ SSH configured")
    
    # Step 3: Set password
    print("\n[3/5] 🔑 Setting SSH password...")
    run(f"echo 'root:{SSH_PASSWORD}' | chpasswd", show=False)
    print(f"      ✓ Password set")
    
    # Add GitHub SSH keys if provided
    if GITHUB_USERNAME:
        print(f"\n[3.5/5] 🔑 Adding SSH keys from GitHub: {GITHUB_USERNAME}")
        result = subprocess.run(f"curl -sf https://github.com/{GITHUB_USERNAME}.keys", 
                               shell=True, capture_output=True, text=True)
        if result.returncode == 0 and result.stdout.strip():
            with open("/root/.ssh/authorized_keys", "a") as f:
                f.write(result.stdout)
            os.chmod("/root/.ssh/authorized_keys", 0o600)
            print(f"      ✓ SSH keys added")
        else:
            print(f"      ⚠️ Could not fetch keys for {GITHUB_USERNAME}")
    
    # Step 4: Start SSH server
    print("\n[4/5] 🖥️  Starting SSH server...")
    run("ssh-keygen -A 2>/dev/null", show=False)
    run("/usr/sbin/sshd", show=False)
    print("      ✓ SSH server running on port 22")
    
    # Step 5: Setup zrok
    print("\n[5/5] 🌐 Configuring zrok...")
    run("zrok disable 2>/dev/null", show=False)
    result = run(f"zrok enable {ZROK_TOKEN} 2>&1", show=False)
    print("      ✓ zrok enabled")
    
    # Print connection info
    print("\n" + "=" * 60)
    print("✅ SETUP COMPLETE!")
    print("=" * 60)
    
    if TUNNEL_MODE == "public":
        print(f"""
📋 CONNECTION INFO:
   SSH User:     root
   SSH Password: {SSH_PASSWORD}
   
🔜 NEXT STEP:
   Run Cell 2 below to start the PUBLIC tunnel.
   Copy the hostname from the output.
   Connect with: ssh root@<hostname>
""")
    else:
        print(f"""
📋 CONNECTION INFO:
   SSH User:     root
   SSH Password: {SSH_PASSWORD}
   
🔜 NEXT STEP:
   Run Cell 2 below to start the PRIVATE tunnel.
   Copy the share token from the output.
   On local machine:
     1. zrok access private <token>
     2. ssh root@localhost -p 9191
""")
    print("=" * 60)

# Run setup
main()

## 🌐 Cell 2: Start Tunnel

**Run this cell and keep it running!**

- **PUBLIC mode**: Look for the hostname (e.g., `abc123.share.zrok.io`)
- **PRIVATE mode**: Look for the share token (e.g., `abc123xyz`)
- Includes auto keep-alive to prevent Kaggle timeout

In [ ]:
# ╔═══════════════════════════════════════════════════════════════╗
# ║              START TUNNEL - KEEP THIS RUNNING                 ║
# ╚═══════════════════════════════════════════════════════════════╝

import subprocess
import threading
import re
import time

print(f"🚀 Starting zrok {TUNNEL_MODE.upper()} tunnel...")
print("=" * 60)

def run_tunnel():
    if TUNNEL_MODE == "public":
        cmd = ['zrok', 'share', 'public', '--backend-mode', 'tcpTunnel', 'localhost:22']
    else:
        cmd = ['zrok', 'share', 'private', '--backend-mode', 'tcpTunnel', 'localhost:22']
    
    process = subprocess.Popen(
        cmd,
        stdout=subprocess.PIPE,
        stderr=subprocess.STDOUT,
        text=True,
        bufsize=1
    )
    
    for line in process.stdout:
        print(line, end='')
        
        # PUBLIC mode - look for hostname
        if TUNNEL_MODE == "public" and 'zrok.io' in line.lower():
            match = re.search(r'https://([a-z0-9]+\.share\.zrok\.io)', line)
            if not match:
                match = re.search(r'([a-z0-9]+\.share\.zrok\.io)', line)
            if match:
                hostname = match.group(1)
                print("\n" + "=" * 60)
                print("🎉 PUBLIC TUNNEL ACTIVE!")
                print("=" * 60)
                print(f"\n📋 CONNECTION INFO:")
                print(f"   Hostname: {hostname}")
                print(f"   SSH Command: ssh root@{hostname}")
                print(f"   Password: {SSH_PASSWORD}")
                print(f"\n💻 VS Code Remote SSH:")
                print(f"   code --remote ssh-remote+root@{hostname} /kaggle/working")
                print(f"\n🐧 Debian 12 Quick Connect:")
                print(f"   python3 connect.py -H {hostname}")
                print(f"   python3 connect.py -H {hostname} --vscode")
                print("\n" + "=" * 60)
        
        # PRIVATE mode - look for share token
        if TUNNEL_MODE == "private" and 'share token' in line.lower():
            match = re.search(r'share token[:\s]+([a-z0-9]+)', line.lower())
            if match:
                token = match.group(1)
                print("\n" + "=" * 60)
                print("🔒 PRIVATE TUNNEL ACTIVE!")
                print("=" * 60)
                print(f"\n📋 CONNECTION INFO:")
                print(f"   Share Token: {token}")
                print(f"   Password: {SSH_PASSWORD}")
                print(f"\n🔗 ON YOUR LOCAL MACHINE:")
                print(f"   Step 1: zrok access private {token}")
                print(f"   Step 2: ssh root@localhost -p 9191")
                print(f"\n🐧 Debian 12 Quick Connect:")
                print(f"   python3 connect.py --private {token}")
                print(f"   python3 connect.py --private {token} --vscode")
                print("\n" + "=" * 60)

# Start tunnel in background thread
tunnel_thread = threading.Thread(target=run_tunnel, daemon=True)
tunnel_thread.start()

# Keep notebook alive
print("⏰ Keeping notebook alive... (Don't close this tab!)")
print("   Press Stop (⬛) button to end the session.\n")

start_time = time.time()
try:
    while True:
        time.sleep(300)  # 5 minutes
        elapsed = int(time.time() - start_time)
        hours, remainder = divmod(elapsed, 3600)
        minutes, seconds = divmod(remainder, 60)
        print(f"[{time.strftime('%H:%M:%S')}] Running for {hours:02d}:{minutes:02d}:{seconds:02d}")
except KeyboardInterrupt:
    print("\n🛑 Tunnel stopped.")

---

## 📖 Local Machine Instructions (Debian 12)

### Quick Setup (One-time)
```bash
# Clone the helper repo
git clone https://github.com/mehtab78/kaggle-vscode-zrok.git
cd kaggle-vscode-zrok/local

# Run setup (installs SSH, sshpass, etc.)
sudo ./setup_debian12.sh
```

### Connect to Kaggle (PUBLIC tunnel)
```bash
# Interactive mode
python3 connect.py

# Direct SSH connection (replace hostname from Cell 2 output)
python3 connect.py -H abc123.share.zrok.io

# Open directly in VS Code
python3 connect.py -H abc123.share.zrok.io --vscode

# Or use plain SSH
ssh root@abc123.share.zrok.io
```

### Connect to Kaggle (PRIVATE tunnel)
```bash
# Connect with share token
python3 connect.py --private abc123xyz

# Open in VS Code
python3 connect.py --private abc123xyz --vscode
```

---

## 🛠️ Optional: Utility Cells

In [ ]:
# Check SSH server status
!ps aux | grep sshd

In [ ]:
# Check zrok status
!zrok status

In [ ]:
# Restart SSH server if needed
!pkill sshd
!/usr/sbin/sshd
print("✅ SSH server restarted")

In [ ]:
# View system info
print("=" * 40)
print("System Information")
print("=" * 40)
!echo "CPU:"; cat /proc/cpuinfo | grep "model name" | head -1
!echo "\nMemory:"; free -h | head -2
!echo "\nGPU:"; nvidia-smi --query-gpu=name,memory.total --format=csv,noheader 2>/dev/null || echo "No GPU"
!echo "\nDisk:"; df -h /kaggle/working

In [ ]:
# Change SSH password (if needed)
import getpass
NEW_PASSWORD = getpass.getpass("Enter new password: ")
!echo "root:{NEW_PASSWORD}" | chpasswd
print("✅ Password changed successfully")